In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
from simpleLSTMWithNNetModel import Model, reloadModel
import tensorflow as tf

In [3]:
# data_train = "data/icd9NotesDataTable_train.csv"
# data_valid = "data/icd9NotesDataTable_valid.csv"
data_train = "data/smallIcd9NotesDataTable_train.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
# vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
# wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
vocab = 'data/icd9Vocab.txt'# vocab for our data
wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
# START_TOKEN = "<s>"
# END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 500
max_grad_norm = 5
codeIdx = 9
textIdx = 6
num_layers = 4
learning_rate = 0.001
training_epochs = 5
batch_size = 1
# total_batches = (xTrain.shape[0]//batch_size)
# print('Total number of batches per epoch %d'%(total_batches))
n_input = 1
# n_steps = 10
n_hidden = 200
# n_classes = helper.n_labels
output_keep_prob = 0.5
input_keep_prob = 1# not sure why there are two but example used 0.5 for output. Check with TA
numLayers = 1
# embeddingSize = embeddings.shape[1]
# print('Embedding size is %d'%(embeddingSize))


In [4]:
output_path = input(prompt = 'Where should models and performances be saved to?')
if modelRunOutputPath == '':
    output_path = 'results/temp'
if not os.path.exists(modelRunOutputPath):
    os.makedirs(modelRunOutputPath)

In [5]:
# helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
#     data_train = data_train, data_valid = data_valid, 
#     maxAllowedNoteLength = maxAllowedNoteLength, 
#     codeIdx = codeIdx, textIdx = textIdx,
#     helperLoadPath = 'results/tempSaveTester/')
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx)
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save('results/tempSaveTester/')# token2id and max length saved to output_path
# handler = logging.FileHandler(log_output)
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
# logging.getLogger().addHandler(handler)

In [6]:
print(helper.icdDict)
# print(helper.tok2id)
print(helper.max_length)

{'18': 2, '123': 4, '4240': 1, '486': 0, '45': 5, '456': 3}
{'horse': 3, 'dog': 1, '<s>': 10, 'CASE:aa': 8, 'UUUNKKK': 9, 'cat': 2, 'the': 5, 'CASE:Aa': 5, 'deadpool': 4, 'CASE:AA': 7, '</s>': 11, 'CASE:aA': 6}
4


In [6]:
print(helper.icdDict)
# print(helper.tok2id)
print(helper.max_length)

{'456': 3, '4240': 1, '45': 5, '486': 0, '123': 4, '18': 2}
{'horse': 3, 'CASE:Aa': 5, 'dog': 1, 'the': 5, 'CASE:AA': 7, 'deadpool': 4, 'CASE:aA': 6, '<s>': 10, '</s>': 11, 'cat': 2, 'UUUNKKK': 9, 'CASE:aa': 8}
4


In [7]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers}
# with open(os.path.join(modelRunOutputPath, 'hyperParamDict.pickle'), 'wb') as handle:
#     pickle.dump(hyperParamsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# hyperParamDict = hyperParamsDict
hyperParamDict

{'EMBED_SIZE': 50,
 'batchSize': 1,
 'inputKeepProb': 1,
 'learningRate': 0.001,
 'maxGradNorm': 5,
 'maxNoteLength': 500,
 'n_hidden': 200,
 'numLayers': 1,
 'outputKeepProb': 0.5,
 'trainingEpochsMax': 5}

In [9]:
helper.n_labels

6

In [10]:
hyperParamDict

{'EMBED_SIZE': 50,
 'batchSize': 1,
 'inputKeepProb': 1,
 'learningRate': 0.001,
 'maxGradNorm': 5,
 'maxNoteLength': 500,
 'n_hidden': 200,
 'numLayers': 1,
 'outputKeepProb': 0.5,
 'trainingEpochsMax': 5}

In [11]:
# tf.reset_default_graph()
# x = tf.placeholder(tf.int32, shape= (None, helper.max_length))
# yTruth = tf.placeholder(tf.int32, shape = (None, helper.n_labels))
# y_steps = tf.placeholder(tf.int32, shape = (None, helper.n_labels))# not sure what this is
# trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))# vector which holds true word
# outputKeepProb = tf.placeholder(tf.float32, shape=())
# inputKeepProb = tf.placeholder(tf.float32, shape=())

In [12]:
# model = Model(xPlaceHolder = x, yPlaceHolder = yTruth, embeddings = embeddings, 
#               hyperParamDict = hyperParamDict)
%load_ext autoreload
%autoreload 2
from simpleLSTMWithNNetModel import Model
tf.reset_default_graph()
model = Model(nColsInput = helper.max_length, nLabels = helper.n_labels,
             embeddings = embeddings, hyperParamDict = hyperParamDict)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(?, 4, 50)
shape of embeddings
(?, 4, 50)
U shape
(200, 6)
bias shape
(6,)
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 4, 50)
cell output size
200
cell state size
(LSTMStateTuple(c=200, h=200),)
output shape
(?, 4, 200)
offset shape
(?, 1)
output shape new shape
(?, 200)
flattened indices shape
(?, 1)
output flattened shape
(?, 200)
output wx + b
(?, 6)
(?, 6)


/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# all_saver = tf.train.Saver()
# tf.add_to_collection('y_last', y_last)
# tf.add_to_collection('x', x)
# tf.add_to_collection('trueWordIdxs', trueWordIdxs)
# tf.add_to_collection('outputKeepProb', outputKeepProb)
# tf.add_to_collection('inputKeepProb', inputKeepProb)
# tf.add_to_collection('pretrainedEmbeddings', pretrainedEmbeddings)
with tf.Session() as session:
#         all_saver.restore(session, 'tempDelete/bestModel.data-00000-of-00001')
    tf.global_variables_initializer().run()
    for i in range(10):
#          for b in range(total_batches):
#             offset = min((b * batch_size), yTrain.shape[0])
#             batch_x = xTrain[offset:(offset + batch_size), :]
#             batch_y = yTrain[offset:(offset + batch_size), :]
#             batchTrueWordIdxs = lastTrueWordIdx_train[offset:(offset + batch_size)]
        _, trainingError = session.run([model.optimize, model.loss_function],
                           feed_dict={model.xPlaceHolder: xTrain, 
                                        model.yPlaceHolder: yTrain, 
                                        model.trueWordIdxs: lastTrueWordIdx_train,
                                        model.outputKeepProb: output_keep_prob, 
                                        model.inputKeepProb: input_keep_prob})
        pred_y = session.run(model.y_last,feed_dict={model.xPlaceHolder: xDev,
                                      model.trueWordIdxs:lastTrueWordIdx_dev,
                                      model.outputKeepProb: 1,
                                      model.inputKeepProb: 1}, ) # must be set to one for predictions.
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('train loss %f'% (trainingError))
        print('test loss %f'%(validLoss))
        print('***********************************************')
        if i == 9:
            print('saving model')
            model.save(session = session, savePath = 'results/tempSaveTester/bestModel')
#         model.save(savePath = 'results/temp/newSave',session = session, 
#                    trueWordIdxs = trueWordIdxs, yTruth = yTrain)

train loss 0.685747
test loss 0.658781
***********************************************
train loss 0.644480
test loss 0.631446
***********************************************
train loss 0.640387
test loss 0.606006
***********************************************
train loss 0.619426
test loss 0.582254
***********************************************
train loss 0.596487
test loss 0.559412
***********************************************
train loss 0.576961
test loss 0.537442
***********************************************
train loss 0.536125
test loss 0.516530
***********************************************
train loss 0.529557
test loss 0.497012
***********************************************
train loss 0.508127
test loss 0.479050
***********************************************
train loss 0.484719
test loss 0.463060
***********************************************
saving model


In [17]:
xPlaceHolder

<tf.Tensor 'Placeholder_13:0' shape=(?, 4) dtype=int32>

In [15]:
len(tf.get_collection('pretrainedEmbeddings'))

3

In [11]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# outputKeepProb = tf.placeholder(tf.float32, shape=())
# inputKeepProb = tf.placeholder(tf.float32, shape=())
# maxLength = helper.max_length
# nLabels = helper.n_labels
# xPlaceHolder = tf.placeholder(tf.int32, shape= (None, maxLength))
# pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')
# yPlaceHolder = tf.placeholder(tf.int32, shape = (None, nLabels))
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
with tf.Session() as session:
#         all_saver.restore(session, 'tempDelete/bestModel.data-00000-of-00001')
    tf.global_variables_initializer().run()
    new_saver = tf.train.import_meta_graph('results/tempSaveTester/bestModel.meta')
    new_saver.restore(session, tf.train.latest_checkpoint('results/tempSaveTester/'))
    y_last = tf.get_collection('y_last')[0]
    outputKeepProb = tf.get_collection('outputKeepProb')[0]
    inputKeepProb = tf.get_collection('inputKeepProb')[0]
    xPlaceHolder = tf.get_collection('xPlaceHolder')[0]
    trueWordIdxs = tf.get_collection('trueWordIdxs')[0]
#     y_last = tf.get_collection('y_last')[0]
    pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')[0]
    for i in range(3):
#         _, trainingError = session.run([model.optimize, model.loss_function],
#                            feed_dict={model.xPlaceHolder: xTrain, 
#                                         model.yPlaceHolder: yTrain, 
#                                         model.trueWordIdxs: lastTrueWordIdx_train,
#                                         model.outputKeepProb: output_keep_prob, 
#                                         model.inputKeepProb: input_keep_prob})
#         print('ok')
        pred_y = session.run(y_last,feed_dict={xPlaceHolder: xDev,
                                      trueWordIdxs:lastTrueWordIdx_dev,
                                      outputKeepProb: 1.0,
                                      inputKeepProb: 1.0}, ) # must be set to one for predictions.
#         print('leggo')
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
#         print('train loss %f'% (trainingError))
        print('test loss %f'%(validLoss))
        print('***********************************************')

test loss 0.463060
***********************************************
test loss 0.463060
***********************************************
test loss 0.463060
***********************************************


In [ ]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# outputKeepProb = tf.placeholder(tf.float32, shape=())
# inputKeepProb = tf.placeholder(tf.float32, shape=())
# maxLength = helper.max_length
# nLabels = helper.n_labels
# xPlaceHolder = tf.placeholder(tf.int32, shape= (None, maxLength))
# pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')
# yPlaceHolder = tf.placeholder(tf.int32, shape = (None, nLabels))
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
with tf.Session() as session:
#         all_saver.restore(session, 'tempDelete/bestModel.data-00000-of-00001')
    tf.global_variables_initializer().run()
    modelDict = reloadModel(session = session,
                            saverCheckPointPath = 'results/tempSaveTester/',
                            saverMetaPath = 'results/tempSaveTester/bestModel.meta')
#     new_saver = tf.train.import_meta_graph('results/tempSaveTester/bestModel.meta')
#     new_saver.restore(session, tf.train.latest_checkpoint('results/tempSaveTester/'))
#     y_last = tf.get_collection('y_last')[0]
#     outputKeepProb = tf.get_collection('outputKeepProb')[0]
#     inputKeepProb = tf.get_collection('inputKeepProb')[0]
#     xPlaceHolder = tf.get_collection('xPlaceHolder')[0]
#     trueWordIdxs = tf.get_collection('trueWordIdxs')[0]
# #     y_last = tf.get_collection('y_last')[0]
#     pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')[0]
    for i in range(3):
#         _, trainingError = session.run([model.optimize, model.loss_function],
#                            feed_dict={model.xPlaceHolder: xTrain, 
#                                         model.yPlaceHolder: yTrain, 
#                                         model.trueWordIdxs: lastTrueWordIdx_train,
#                                         model.outputKeepProb: output_keep_prob, 
#                                         model.inputKeepProb: input_keep_prob})
#         print('ok')
#         pred_y = session.run(y_last,feed_dict={xPlaceHolder: xDev,
#                                       trueWordIdxs:lastTrueWordIdx_dev,
#                                       outputKeepProb: 1.0,
#                                       inputKeepProb: 1.0}, ) # must be set to one for predictions.
#         print('leggo')
        pred_y = session.run(modelDict['y_last'],feed_dict={modelDict['xPlaceHolder']: xDev,
                                      modelDict['trueWordIdxs']:lastTrueWordIdx_dev,
                                      modelDict['outputKeepProb']: 1.0,
                                      modelDict['inputKeepProb']: 1.0}, ) 
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
#         print('train loss %f'% (trainingError))
        print('test loss %f'%(validLoss))
        print('***********************************************')

In [ ]:
# import functools
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data



# def LSTM(x, weight, bias, trueWordIdxs, outputKeepProb, inputKeepProb):
#     cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,state_is_tuple = True)
#     cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = outputKeepProb, 
#                                          input_keep_prob = inputKeepProb)
#     cell = tf.contrib.rnn.MultiRNNCell(cells=[cell] * num_layers, state_is_tuple=True)
# #     If we ever wanna get fancy we can try the above.
#     print(type(cell))
#     print(cell)
#     print(type(x))
#     print(x.get_shape())
#     print('cell output size')
#     print(cell.output_size)
#     print('cell state size')
#     print(cell.state_size)

#     output, state = tf.nn.dynamic_rnn(cell = cell, inputs = x, dtype = tf.float32)
#     print('output shape')
#     print(output.get_shape())
#     # new code
#     offset = tf.expand_dims(tf.range(0, batch_size, dtype = tf.int32)*helper.max_length, 1)
#     offset = tf.expand_dims(tf.range(0, tf.shape(x)[0], dtype = tf.int32)*helper.max_length, 1)
#     print('offset shape')
#     print(offset.get_shape())
#     output = tf.reshape(output,[-1, n_hidden]) # collapses the 3d matrix into a 2d
#     # matrix where all matrices are stacked on top of eachother
#     print('output shape new shape')
#     print(output.get_shape())
#     flattened_indices = trueWordIdxs + offset
#     print('flattened indices shape')
#     print(flattened_indices.get_shape())
#     output_flattened = tf.gather(output, flattened_indices)
#     output_flattened = tf.reshape(output_flattened, [-1, n_hidden])
#     print('output flattened shape')
#     print(output_flattened.get_shape())
#     output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
#     print('output wx + b')
#     print(output_logits.get_shape())
#     return output_logits


# def doublewrap(function):
#     """
#     A decorator decorator, allowing to use the decorator to be used without
#     parentheses if not arguments are provided. All arguments must be optional.
#     """
#     @functools.wraps(function)
#     def decorator(*args, **kwargs):
#         if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
#             return function(args[0])
#         else:
#             return lambda wrapee: function(wrapee, *args, **kwargs)
#     return decorator


# @doublewrap
# def define_scope(function, scope=None, *args, **kwargs):
#     """
#     A decorator for functions that define TensorFlow operations. The wrapped
#     function will only be executed once. Subsequent calls to it will directly
#     return the result so that operations are added to the graph only once.
#     The operations added by the function live within a tf.variable_scope(). If
#     this decorator is used with arguments, they will be forwarded to the
#     variable scope. The scope name defaults to the name of the wrapped
#     function.
#     """
#     attribute = '_cache_' + function.__name__
#     name = scope or function.__name__
#     @property
#     @functools.wraps(function)
#     def decorator(self):
#         if not hasattr(self, attribute):
#             with tf.variable_scope(name, *args, **kwargs):
#                 setattr(self, attribute, function(self))
#         return getattr(self, attribute)
#     return decorator


# class Model:

#     def __init__(self, xPlaceHolder, yPlaceHolder, embeddings, hyperParamDict):
#         self.xPlaceHolder = xPlaceHolder
#         self.yPlaceHolder = yPlaceHolder
# #         self.embeddings = embeddings
#         self.pretrainedEmbeddings = tf.Variable(embeddings)
#         self.hyperParamDict = hyperParamDict
#         self.y_last
#         self.loss_function
#         self.optimize

#     @define_scope#(initializer=tf.contrib.slim.xavier_initializer())
#     def y_last(self):
# #         print(wtfStr)
#         n_classes = int(self.yPlaceHolder.shape[1])
#         x = self.xPlaceHolder
#         U = tf.get_variable(name = 'U', 
#                             shape = (self.hyperParamDict['n_hidden'], n_classes), 
#                         initializer = tf.contrib.layers.xavier_initializer())
#         bias = tf.get_variable(name = 'bias', shape = [n_classes], 
#                                initializer = tf.constant_initializer(0))
# #         pretrainedEmbeddings = tf.Variable(self.embeddings)
#         wordEmbeddings = tf.nn.embedding_lookup(params = self.pretrainedEmbeddings, ids = x)
#         print(wordEmbeddings.get_shape())
#         print('shape of embeddings')
#         print(wordEmbeddings.get_shape())
#         print('U shape')
#         print(U.get_shape())
#         print('bias shape')
#         print(bias.get_shape())
#         y_last = LSTM(wordEmbeddings,U,bias, trueWordIdxs, outputKeepProb, inputKeepProb)# TODO is y_last the correct thing to return?
#         print(y_last.get_shape())
#         return(y_last)

#     @define_scope
#     def optimize(self):
#         optimizer = tf.train.AdamOptimizer(
#             learning_rate = self.hyperParamDict['learning_rate'])
#         # optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate) dis from hw3
#         tvars = tf.trainable_variables()
#         grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss_function, tvars), 
#                                           max_grad_norm)
#         gradientVars = zip(grads, tvars)
#         train_op = optimizer.apply_gradients(gradientVars)
#         return(train_op)
# #         logprob = tf.log(self.prediction + 1e-12)
# #         cross_entropy = -tf.reduce_sum(self.label * logprob)
# #         optimizer = tf.train.RMSPropOptimizer(0.03)
# #         return optimizer.minimize(cross_entropy)

#     @define_scope
#     def loss_function(self):
#         batchError = tf.nn.sigmoid_cross_entropy_with_logits(logits = self.y_last, 
#                                                  labels = tf.cast(self.yPlaceHolder, tf.float32))
#         loss_function = tf.reduce_mean(batchError)
#         return(loss_function)
    
#     def save(self, savePath, session):
#         """
#         Saves the computation map so that it can be loaded later for use
#         in computation, or training.
#         Might remove stuff needed for training (x, trueWordsIdx)
#         """
#         all_saver = tf.train.Saver()
#         tf.add_to_collection('y_last', self.y_last)
#         tf.add_to_collection('xPlaceHolder', self.xPlaceHolder)# this just makes loading easier. Will have to
#         # stop doing this in future because not necessary
#         tf.add_to_collection('trueWordIdxs', trueWordIdxs)
#         tf.add_to_collection('outputKeepProb', outputKeepProb)
#         tf.add_to_collection('inputKeepProb', inputKeepProb)
#         tf.add_to_collection('pretrainedEmbeddings', self.pretrainedEmbeddings)
#         tf.add_to_collection('yPlaceHolder', self.yPlaceHolder)
#         all_saver.save(session, savePath)
        
        
# #     def load(session, saverMetaPath, saverCheckPointPath):
# # #         saveMetaPath = 'results/temp/bestModel.meta'
# # #         saverCheckPointPath = 'results/temp/'
# #         new_saver = tf.train.import_meta_graph(saverMetaPath)
# #         new_saver.restore(session, tf.train.latest_checkpoint(saverCheckPointPath))
# #         xPlaceHolder = tf.get_collection('xPlaceHolder')[0]
# #         trueWordIdxs = tf.get_collection('trueWordIdxs')[0]
# #         outputKeepProb = tf.get_collection('outputKeepProb')[0]
# #         inputKeepProb = tf.get_collection('inputKeepProb')[0]
# #         pretrainedEmbeddings = tf.get_collection('pretrainedEmbeddings')[0]
# #         yTruth = tf.get_collection('yTruth')[0]
# #         y_last = tf.get_collection('y_last')[0]
# #         self.xPlaceHolder = xPlaceHolder
# #         self.yPlaceHolder = yPlaceHolder
# # #         self.embeddings = embeddings
# #         self.pretrainedEmbeddings = tf.Variable(embeddings)
# #         self.hyperParamDict = hyperParamDict
# #         self.y_last
# #         self.loss_function
# #         self.optimize
# #         return(0)
